In [1]:
import boto3
import copy
import io
import ipywidgets as widgets
import ipyleaflet as leaflet
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import calendar
import datetime
import geojson
import pyproj
import pytz
import shapely
from shapely.ops import cascaded_union
import time

# local imports
from helpers import *
from map_elements import *

import warnings
warnings.filterwarnings('ignore')

In [2]:
%%html
<style>
.jupyter-widgets.widget-tab > .p-TabBar .p-TabBar-tab {
    flex: 0 1 300px
}
.lbl_bg {
    width: auto;
    background-color: #F0F0F0;
    border-radius: 4px;
}
.filter_box {
    width: auto;
    border-style: solid;
#     background-color: #F0F0F0;
    border-radius: 4px;
    border-color: #F0F0F0;
    border-width: 4px;
    height: 603px;
}
.filter_box2 {
    width: auto;
    border-style: solid;
#     background-color: #F0F0F0;
    border-radius: 4px;
    border-color: #F0F0F0;
    border-width: 4px;
    height: 900px;
}

.filter_box3 {
    width: auto;
    border-style: solid;
#     background-color: #F0F0F0;
    border-radius: 4px;
    border-color: #F0F0F0;
    border-width: 4px;
    height: 403px;
}
</style>

## Connect to AWS

In [3]:
with open("../creds/aws_creds.json") as f:
    aws_creds = json.loads(f.read())
    
client = boto3.client('s3',
                      aws_access_key_id=aws_creds["access_key_id"],
                      aws_secret_access_key=aws_creds["access_key_secret"])

s3 = boto3.resource('s3',
         aws_access_key_id = aws_creds["access_key_id"],
         aws_secret_access_key = aws_creds["access_key_secret"])

## Read in SatCat data

In [4]:
bucket = 'afv-scenario'
key = 'data-products/Reference/sat_cat_combined_20201229.parquet'
satcat = client.get_object(Bucket=bucket, Key=key)
satcat = pd.read_parquet(io.BytesIO(satcat['Body'].read()))
satcat = satcat[satcat['NORAD_CAT_ID'].isin(get_existing_satellites(s3))]
satcat = satcat.dropna(subset = ['COUNTRY'])
satcat.head()

,INTLDES,NORAD_CAT_ID,OBJECT_TYPE,SATNAME,COUNTRY,LAUNCH,SITE,DECAY,PERIOD,INCLINATION,...,RCSVALUE,RCS_SIZE,FILE,LAUNCH_YEAR,LAUNCH_NUM,LAUNCH_PIECE,CURRENT,OBJECT_NAME,OBJECT_ID,OBJECT_NUMBER
1879,2015-008A,40392,PAYLOAD,PROGRESS-M 26M,CIS,2015-02-17,TTMTR,2015-08-14,92.59,51.65,...,0,None,5799,2015,8,A,Y,PROGRESS-M 26M,2015-008A,40392
1880,2015-016A,40542,PAYLOAD,SOYUZ-TMA 16M,CIS,2015-03-27,TTMTR,2015-09-12,92.60,51.65,...,0,None,5799,2015,16,A,Y,SOYUZ-TMA 16M,2015-016A,40542
3081,1973-084D,6939,ROCKET BODY,SL-6 R/B(2),CIS,1973-11-02,PKMTR,2015-06-29,92.02,60.93,...,0,LARGE,5942,1973,84,D,Y,SL-6 R/B(2),1973-084D,6939
3678,1992-084B,22254,ROCKET BODY,ARIANE 42P+ R/B,FR,1992-12-01,FRGUI,2015-05-17,88.77,6.64,...,0,LARGE,7337,1992,84,B,Y,ARIANE 42P+ R/B,1992-084B,22254
3858,1992-040D,22020,ROCKET BODY,SL-6 R/B(2),CIS,1992-07-08,PKMTR,2015-06-29,92.34,60.83,...,0,LARGE,7337,1992,40,D,Y,SL-6 R/B(2),1992-040D,22020


## Create Map 1

In [5]:
warnings.filterwarnings('ignore')

# Map 1 Observer
def on_map1_selected(change):    
        
    # Safeguards
    if end_date.value < start_date.value:
        progress_text2.value = "Please choose a start time that is before the end time and reset the map."
        return  
    
    # Clear the current map, re-add the basemap and reset progress variables
    m.clear_layers()
    m.add_layer(basemap_layer)
    vessels_missed_number.value, vessels_seen_number.value, area_covered_number.value = "", "", ""
    
    # Read in selected FoV data
    progress_text1.value = "Reading in " + satellite_selector.value + " field of view data..."
    fov, norad_id = pull_fov(satellite_selector, satcat, client)
    fov['Time'] = pd.to_datetime(fov['Time'], infer_datetime_format=True, utc = True)
    
    # Subset the dataframe to the selected data and get the satellite view polygons
    progress_text1.value = "Subsetting " + satellite_selector.value + " field of view data..."
    new_df = fov[(fov['Time'] >= start_date.value) & (fov['Time'] <= end_date.value)]
    new_df = new_df[new_df['Satellite_NORAD_ID'] == norad_id]
    
    # Convert new_df to a geodataframe
    new_df['geometry'] = new_df['FOV'].apply(shapely.wkt.loads)
    new_df = gpd.GeoDataFrame(new_df, geometry='geometry')

    # Get a list os plygon geometries and FoV confidence levels for plotting
    polys, confidence = list(new_df.geometry), list(new_df.Confidence)
        
    # Fix the polygon geometry
    new_df.geometry = fix_geometry(polys)
    
    # Calculate total FoV coverage area and update message
    tmp_for_area = copy.deepcopy(new_df)  
    ortho = pyproj.CRS.from_proj4("+proj=ortho +lat_0=60.00 +lon_0=23.0000 +x_0=0 +y_0=0 +a=6370997 +b=6370997 +units=m +no_defs")
    wgs84 = pyproj.CRS.from_proj4("+proj=utm +zone=51 +datum=WGS84 +units=m +no_defs +ellps=WGS84 +towgs84=0,0,0")
    tmp_for_area.geometry.crs = ortho
    tmp_for_area.geometry = tmp_for_area.geometry.to_crs(wgs84)
    tmp_for_area["area"] = tmp_for_area['geometry'].area
    total_area = tmp_for_area["area"].sum() ** 2
    area_covered_number.value = "<center>" + str(total_area)


    count = 0
    for poly in polys:
        
        # Get exterior coords of current FoV poly
        x, y = poly.exterior.xy
                
        # Try block here to prevent some of the polygons that are disfigured due to projection from being plotted
        try:
            p_list = []
            for j in range(0, len(x)):
                p_list.append(shapely.geometry.Point(y[j], x[j]))
            poly = shapely.geometry.Polygon([[p.x, p.y] for p in p_list])   
            test = poly.intersection(poly) 
            
            # Color the FoV by the confidence level
            if confidence[count] == "High":
                polygon = leaflet.Polygon(locations = list(poly.exterior.coords), color = 'green', fill_color = 'green', weight = 1)
            elif confidence[count] == "Low":
                polygon = leaflet.Polygon(locations = list(poly.exterior.coords), color = 'red', fill_color = 'red', weight = 1)
                
            # Sleep for a second to see orbit progression
            time.sleep(1)
            m.add_layer(polygon)
        except shapely.geos.TopologicalError:
            pass
        count += 1


    # Read in AIS data from s3 and subset to only a few ships for rendering purposes
    progress_text1.value = "Grabbing AIS data..."
    subset = get_ais(year_selector.value, client)
    subset['BaseDateTime'] = pd.to_datetime(subset['BaseDateTime'], infer_datetime_format=True, utc = True)
    subset = subset[(subset['BaseDateTime'] >= start_date.value) & (subset['BaseDateTime'] <= end_date.value)]
    subset = subset[~subset.index.duplicated()]
    subset = subset[0:500] # Grab a smaller subset for rendering's sake


    # Get all of the vessels within the selected time period and find which ones overlap with the drawn polygon (gdp.clip)
    geometry = [shapely.geometry.Point(xy) for xy in zip(subset.LAT, subset.LON)]
    gdf = gpd.GeoDataFrame(subset, geometry = geometry)
    progress_text1.value = "Clipping AIS data to " + satellite_selector.value + " field of view." 
    points_clip = gpd.clip(gdf, new_df)
    
    # Fill in information about vessels seen/not seen
    num_vessels_missed = len(gdf) - len(points_clip)
    vessels_missed_number.value = "<center>" + str(num_vessels_missed)
    vessels_seen_number.value = "<center>" + str(len(points_clip))
    
    # Alert users the mapping is done
    progress_text1.value = "Finished."
    
    # Plot vessels that overlap with FoV
    count, cluster_points = 1, []
    for col, row in points_clip.iterrows():
        cluster_points.append(create_circle_marker(row))
        progress_text1.value = "Plotting vessel " + str(count) + " out of " + str(len(points_clip))
        count += 1
        
    # Add the vessels to the map as a MarkerCluster layer
    marker_cluster = leaflet.MarkerCluster(markers = cluster_points)
    m.add_layer(marker_cluster)
    
    

def get_days(change):
    return get_days_observer(year_selector, month_selector)

def update_day(change):
    update_day_observer(datetime_slider, year_selector, month_selector, day_selector)

def filter_satellites(change):
    filter_satellites_observer(satcat, country_selecter, satellite_selector)

def update_year(change):
    dates = initialize_date_options(year_selector, month_selector, day_selector)
    start_date.options = dates
    end_date.options = dates
    end_date.value = dates[-2]
    day_selector.options = get_days_observer(year_selector, month_selector)
    
def filter_month(change):
    filter_month_observer(month_selector, year_selector, day_selector, start_date, end_date)
    

# Create Map 1 Elements reliant on data
country_selecter = widgets.Dropdown(options = sorted(satcat['COUNTRY'].unique()), value = 'CIS', layout=widgets.Layout(width='auto'))
intial_sats = satcat[satcat['COUNTRY'] == country_selecter.value]
satellite_selector = widgets.Dropdown(options = sorted(intial_sats['SATNAME'].unique()), value = 'COSMOS 1346', layout=widgets.Layout(width='auto'))
area_covered_text = widgets.HTML(value = ("<br><br><center> <b>Area covered in m&sup2 by " + satellite_selector.value + " in chosen time range</b>"))
day_selector = widgets.Dropdown(options = get_days_observer(year_selector, month_selector), description = 'Day: ')
dates = initialize_date_options(year_selector, month_selector, day_selector)
vessels_missed_text = widgets.HTML(value="<br><br><b> <center> Vessels out of view of " + satellite_selector.value + " during selected time range</b>")
vessels_seen_text = widgets.HTML(value="<br><br><b> <center> Vessels in view of " + satellite_selector.value + " during selected time range</b>")
start_date = widgets.Dropdown(options = dates, value = dates[191], description = 'Start time:')
end_date = widgets.Dropdown(options = dates, value = dates[201], description = 'End time:')
progress_text1 = widgets.HTML(value = "")



# Create blank starting map
m = leaflet.Map(center=(0, 0), zoom=1)
m.layout.height = '600px'
legend = leaflet.LegendControl({"High":"green", "Low":"red"}, name="Field of View Confidence", position="topright")
m.add_control(legend)


# Once a dropdown option is selected, change the data on the map
reset_map1_button.on_click(on_map1_selected)
country_selecter.observe(filter_satellites)
year_selector.observe(update_year)
month_selector.observe(filter_month)
day_selector.observe(update_year)


# Create visuzalization
viz1 = widgets.GridspecLayout(4, 3, height='850px', layout = widgets.Layout(grid_gap='15px 5px'))
viz1[1:, 1:] = m
viz1[1:4, 0] = widgets.VBox([country_select_text, country_selecter, sat_selector_label, satellite_selector, 
                             year_select_text, year_selector, month_selector, day_selector, date_slider_text, 
                             start_date, end_date, br, reset_map1_button, br, progress_text1]).add_class('filter_box')
viz1[0, 0] = widgets.VBox([vessels_missed_text, vessels_missed_number]).add_class('lbl_bg')
viz1[0, 1] = widgets.VBox([vessels_seen_text, vessels_seen_number]).add_class('lbl_bg')
viz1[0, 2] = widgets.VBox([area_covered_text, area_covered_number,]).add_class('lbl_bg')
viz1

GridspecLayout(children=(Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

TopologyException: Input geom 0 is invalid: Self-intersection at or near point -67.384508752972195 39.081460392942425 at -67.384508752972195 39.081460392942425
TopologyException: Input geom 0 is invalid: Self-intersection at or near point -49.525205187477042 -173.53530051540292 at -49.525205187477042 -173.53530051540292
TopologyException: Input geom 0 is invalid: Self-intersection at or near point -72.81594639783458 -149.23352872401156 at -72.81594639783458 -149.23352872401156
TopologyException: Input geom 0 is invalid: Self-intersection at or near point -48.423374162397749 -138.7627734441877 at -48.423374162397749 -138.7627734441877
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 53.560156546336884 -106.30751406780986 at 53.560156546336884 -106.30751406780986
TopologyException: Input geom 0 is invalid: Self-intersection at or near point 67.422435927966404 -47.708433639631458 at 67.422435927966404 -47.708433639631458
TopologyException: Input geom 0 is inv

## Create Map 2

Please note that this has been tested on a limited set of configurations (e.g. MacOS), it may not work properly on others.

In [6]:
def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map""" 
    
    # Safeguards
    if end_date2.value < start_date2.value:
        progress_text2.value = "Please choose a start time that is before the end time and reset the map."
        return
    elif len(us_satellite_selector2.value) > 5:
        progress_text2.value = "Please choose 5 or less satellites to plot and reset the map."
        return
    elif len(satellite_selector2.value) > 5:
        progress_text2.value = "Please choose 5 or less satellites to plot and reset the map."
        return
    elif len(us_satellite_selector2.value) == 0:
        progress_text2.value = "Please choose at least 1 US satellites."
        return
    elif len(satellite_selector2.value) == 0:
        progress_text2.value = "Please choose at least 1 foreign satellite."
        return        
    
    # Clear the current map and re-add the basemap
    dc.clear()
    m2.clear_layers()
    m2.add_layer(basemap_layer)
            
    # Add the most recently drawn polygon to the map, recenter and reset the zoom
    bounds = fix_coordinates(
        geom = dc.last_draw)
    center = np.array(shapely.geometry.Polygon([[p[0], p[1]] for p in bounds]).centroid)    
    m2.center = (center[0], center[1])
    last_drawn = leaflet.Polygon(locations = bounds, color = "gray", fill_color = "gray", weight = 4)#.to_crs("EPSG:3395")
    m2.add_layer(last_drawn)
    m2.zoom = 2
    
    # Read in AIS data from s3 and subset to only a few ships for rendering purposes
    progress_text2.value = "Grabbing AIS data..."
    subset = get_ais(year_selector2.value, client)
    subset['BaseDateTime'] = pd.to_datetime(subset['BaseDateTime'], infer_datetime_format=True, utc = True)
    subset = subset[(subset['BaseDateTime'] >= start_date2.value) & (subset['BaseDateTime'] <= end_date2.value)]
    subset = subset[~subset.index.duplicated()]
    subset = subset[0:500]
    geometry = [shapely.geometry.Point(xy) for xy in zip(subset.LAT, subset.LON)]
    gdf = gpd.GeoDataFrame(subset, geometry = geometry)
        
    '''FOREIGN SATELLITES'''
    progress_text2.value = "Grabbing " + ", ".join(satellite_selector2.value) + " field of view data."
    # Get FoV's for selected foreign satellites
    fov = get_sat_fovs(country_selecter2, satcat, satellite_selector2, client)
        
    # Subset the dataframe to the selected data and get the satellite view polygons (new_df)
    fov['Time'] = pd.to_datetime(fov['Time'], infer_datetime_format=True, utc = True)
    new_df = fov[(fov['Time'] >= start_date2.value) & (fov['Time'] <= end_date2.value)]
    new_df['geometry'] = new_df['FOV'].apply(shapely.wkt.loads)
    new_df = gpd.GeoDataFrame(new_df, geometry='geometry')

    # Fix the geometry of the FoV's
    new_df.geometry = fix_geometry(new_df.geometry)
    new_df = new_df.loc[new_df['geometry'].is_valid, :]
    view_clip = gpd.clip(new_df, shapely.geometry.Polygon(bounds))

    # Calculate area covered by foreign satellites
    tmp_poly_covered = gpd.GeoDataFrame(crs={'init': 'epsg:4326'}, geometry = [cascaded_union(view_clip['geometry'])])
    tmp_poly_covered['area'] = tmp_poly_covered.geometry.area
    area_covered_chosen = tmp_poly_covered['area'].sum()
                                                                               
    # Calculate percentage of drawn area 
    tmp_poly = gpd.GeoDataFrame(crs={'init': 'epsg:4326'}, geometry = [shapely.geometry.Polygon(bounds)])
    tmp_poly['area'] = tmp_poly.geometry.area
    area_total_chosen = tmp_poly['area'].sum()

    # Calculate percentage of drawn area covered by foreign satellites
    perc = round((area_covered_chosen / area_total_chosen) * 100, 2)
    other_area_cov_num.value = "<center>" + str(perc) + "%"
    
    # For each satellite that overlaps with the drawn area, plot the union of its overlapping FoV's
    progress_text2.value = "Plotting " + country_selecter2.value + " coverage within selected area."
    union = cascaded_union(view_clip['geometry']) 
    if union.geom_type == 'Polygon':
        union_to_plot = leaflet.Polygon(locations = list(union.exterior.coords), color = "red", fill_color="red", weight = 2)
        m2.add_layer(union_to_plot)
    else:
        for poly in union:
            union_to_plot = leaflet.Polygon(locations = list(poly.exterior.coords), color = "red", fill_color="red", weight = 2)
            m2.add_layer(union_to_plot)
        
        
    # Get all of the vessels within the selected time period and find which ones overlap with the drawn polygon (gdp.clip)
    progress_text2.value = "Clipping AIS data to field of view." 
    if union.geom_type == 'Polygon':
        
        points_clip = gpd.clip(gdf, union)
        oth_num_vssels_seen.value = "<center>" + str(len(points_clip))
            
        # Plot vessels that overlap with FoV
        count, cluster_points = 0, []
        for col, row in points_clip.iterrows():
            cluster_points.append(create_colored_circle_marker(row, "red"))
            progress_text2.value = "Plotting vessel " + str(count) + " out of " + str(len(points_clip))
            count += 1

        # Add the vessels to the map as a MarkerCluster layer
        marker_cluster = leaflet.MarkerCluster(markers = cluster_points)
        m2.add_layer(marker_cluster)
    
    else:
        count, cluster_points = 0, []
        for poly in union:
            
            points_clip = gpd.clip(gdf, poly)
            oth_num_vssels_seen.value = "<center>" + str(len(points_clip))

            # Plot vessels that overlap with FoV
            for col, row in points_clip.iterrows():
                cluster_points.append(create_colored_circle_marker(row, "red"))
                progress_text2.value = "Plotting vessel " + str(count) + " out of " + str(len(points_clip))
                count += 1

        # Add the vessels to the map as a MarkerCluster layer
        marker_cluster = leaflet.MarkerCluster(markers = cluster_points)
        m2.add_layer(marker_cluster)
    
    '''US SATELLITES'''
    # Get FoV's for selected foreign satellites
    progress_text2.value = "Grabbing " + ", ".join(us_satellite_selector2.value) + " field of view data."
    fov = get_sat_fovs(country_selecter2, satcat, us_satellite_selector2, client)
        
    # Subset the dataframe to the selected data and get the satellite view polygons (new_df)
    fov['Time'] = pd.to_datetime(fov['Time'], infer_datetime_format=True, utc = True)
    new_df = fov[(fov['Time'] >= start_date2.value) & (fov['Time'] <= end_date2.value)]
    new_df['geometry'] = new_df['FOV'].apply(shapely.wkt.loads)
    new_df = gpd.GeoDataFrame(new_df, geometry='geometry')

    # Fix the geometry of the FoV's
    new_df.geometry = fix_geometry(new_df.geometry)
    new_df = new_df.loc[new_df['geometry'].is_valid, :]
    view_clip = gpd.clip(new_df, shapely.geometry.Polygon(bounds))

    # Calculate area covered by foreign satellites
    tmp_poly_covered = gpd.GeoDataFrame(crs={'init': 'epsg:4326'}, geometry = [cascaded_union(view_clip['geometry'])])
    tmp_poly_covered['area'] = tmp_poly_covered.geometry.area
    area_covered_chosen = tmp_poly_covered['area'].sum()
                                                                               
    # Calculate percentage of drawn area 
    tmp_poly = gpd.GeoDataFrame(crs={'init': 'epsg:4326'}, geometry = [shapely.geometry.Polygon(bounds)])
    tmp_poly['area'] = tmp_poly.geometry.area
    area_total_chosen = tmp_poly['area'].sum()

    # Calculate percentage of drawn area covered by foreign satellites
    perc = round((area_covered_chosen / area_total_chosen) * 100, 2)
    us_area_cov_num.value = "<center>" + str(perc) + "%"
    
    # For each satellite that overlaps with the drawn area, plot the union of its overlapping FoV's
    progress_text2.value = "Plotting " + ", ".join(us_satellite_selector2.value) + " coverage within selected area."
    union = cascaded_union(view_clip['geometry']) 
    if union.geom_type == 'Polygon':
        union_to_plot = leaflet.Polygon(locations = list(union.exterior.coords), color = "green", fill_color="green", weight = 2)
        m2.add_layer(union_to_plot)
    else:
        for poly in union:
            union_to_plot = leaflet.Polygon(locations = list(poly.exterior.coords), color = "green", fill_color="green", weight = 2)
            m2.add_layer(union_to_plot)
            
    # Get all of the vessels within the selected time period and find which ones overlap with the drawn polygon (gdp.clip)
    progress_text2.value = "Clipping AIS data to field of view." 
    if union.geom_type == 'Polygon':
        
        points_clip = gpd.clip(gdf, union)
        us_num_vssels_seen.value = "<center>" + str(len(points_clip))
            
        # Plot vessels that overlap with FoV
        count, cluster_points = 0, []
        for col, row in points_clip.iterrows():
            cluster_points.append(create_colored_circle_marker(row, "green"))
            progress_text2.value = "Plotting vessel " + str(count) + " out of " + str(len(points_clip))
            count += 1

        # Add the vessels to the map as a MarkerCluster layer
        marker_cluster = leaflet.MarkerCluster(markers = cluster_points)
        m2.add_layer(marker_cluster)
    
    else:
        count, cluster_points = 0, []
        for poly in union:
            points_clip = gpd.clip(gdf, poly)
            us_num_vssels_seen.value = "<center>" + str(len(points_clip))

            # Plot vessels that overlap with FoV
            for col, row in points_clip.iterrows():
                cluster_points.append(create_colored_circle_marker(row, "green"))
                progress_text2.value = "Plotting vessel " + str(count) + " out of " + str(len(points_clip))
                count += 1

        # Add the vessels to the map as a MarkerCluster layer
        marker_cluster = leaflet.MarkerCluster(markers = cluster_points)
        m2.add_layer(marker_cluster)
        
        
    progress_text2.value = "Finished."


    
def filter_satellites2(change):
    filter_satellites_obserrver(satcat, country_selecter2, satellite_selector2)
    
def reset_map(change):
    reset_map2_observer(dc, m2, basemap_layer, progress_text2, us_num_vssels_seen, us_area_cov_num, other_area_cov_num, oth_num_vssels_seen)
    
def update_year2(change):
    dates2 = initialize_date_options(year_selector2, month_selector2, day_selector2)
    start_date2.options = dates2
    end_date2.options = dates2
    end_date2.value = dates2[-2]
    day_selector2.options = get_days_observer(year_selector2, month_selector2)
    
    
def filter_month2(change):
    filter_month_observer(month_selector2, year_selector2, day_selector2, start_date2, end_date2)
    
    
    
# Intialize map 2 widgets
country_select_text = widgets.HTML(value = "<b>Select the country or entity that owns the satellite: </b>")
country_selecter2 = widgets.Dropdown(options = satcat['COUNTRY'].unique(), layout=widgets.Layout(width='auto'))
satellite_selector2_text = widgets.HTML("<b>Select up to 5 foreign satellites:</b>")
other_sats = satcat[satcat['COUNTRY'] == country_selecter2.value]
satellite_selector2 = widgets.SelectMultiple(options = sorted(other_sats['SATNAME'].unique()), value = ['AIST 1', 'AIST 2', 'COSMOS 1346'], rows=10, disabled=False)
us_satellite_selector2_text = widgets.HTML("<b>Select up to 5 US satellites:</b>")
usa = satcat[satcat['COUNTRY'] == 'US']
us_satellite_selector2 = widgets.SelectMultiple(options = sorted(usa['SATNAME'].unique()), value = ['50 SAT', 'ACRIMSAT', 'AEROCUBE 2'], rows=10, disabled=False)
year_select_text = widgets.HTML(value = "<b>Select the year to show FoV's for: </b>")
year_selector2 = widgets.Dropdown(options = [2015, 2016, 2017], description = 'Year: ')
month_selector2 = widgets.Dropdown(options = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], description = 'Month:')
day_selector2 = widgets.Dropdown(options = get_days_observer(year_selector2, month_selector2), description = 'Day: ')
dates2 = initialize_date_options(year_selector2, month_selector2, day_selector2)
instructions = widgets.HTML(value = "<b>Draw a polygon on the map and select <br> a time range to see vessels viewed by satellites <br> within the selected area. </b>")
start_date2 = widgets.Dropdown(description = "Start time: ", options = dates2, value = dates[0])
end_date2 = widgets.Dropdown(description = "End time: ", value = dates[239], options = dates2)
other_area_cov_text = widgets.HTML(value="<center> <br> <b> Percentage of drawn area covered by foreign satellites: </b>")
progress_text2 = widgets.HTML(value = "")
us_num_vssels_seen_text = widgets.HTML(value = "<br><b><center>Number of ships seen by US satellites: </b>")
us_num_vssels_seen = widgets.HTML(value = "")
oth_num_vssels_seen_text = widgets.HTML(value = "<br><b><center>Number of ships seen by foreign satellites: </b>")
oth_num_vssels_seen = widgets.HTML(value = "")

 
# Initialize Map 2        
m2 = leaflet.Map(basemap = basemap_layer, center=(0, 0), zoom=1)#, crs = projections.EPSG3857)
dc = leaflet.DrawControl(rectangle={'shapeOptions': {'color': '#0000FF'}}, marker={},  circlemarker={},  polyline = {},polygon = {}, circle = {})
m2.add_control(dc)
m2.layout.height = '600px'
header2 = widgets.HTML("<h2> Given a time and location, what vessels were in the field of view of a satellite?</h2>", layout=widgets.Layout(height='auto'))
legend = leaflet.LegendControl({"Foreign satellites":"red", "US Satellites":"green", "Area of Interest":'gray'}, name="Legend", position="topright")
m2.add_control(legend)


# Map Observers
dc.on_draw(handle_draw)
reset_map_button.on_click(reset_map)
country_selecter2.observe(filter_satellites2)
month_selector2.observe(filter_month2)
year_selector2.observe(update_year2)
day_selector2.observe(update_year2)


# Create visuzalization
viz2 = widgets.GridspecLayout(6, 4, height='1000px', layout = widgets.Layout(grid_gap='15px 5px'))
viz2[1:4, 1:] = m2
viz2[1:4, 0] = widgets.VBox([country_select_text, country_selecter2, satellite_selector2_text, satellite_selector2, 
                             us_satellite_selector2_text, us_satellite_selector2, year_select_text, year_selector2, 
                             month_selector2, day_selector2, date_slider_text2, start_date2, end_date2, reset_map_button, 
                             loading_text, progress_text2]).add_class('filter_box2')
viz2[0, 0] = widgets.VBox([us_num_vssels_seen_text, us_num_vssels_seen]).add_class('lbl_bg')
viz2[0, 1] = widgets.VBox([us_area_cov_text, us_area_cov_num]).add_class('lbl_bg')
viz2[0, 2] = widgets.VBox([other_area_cov_text, other_area_cov_num]).add_class('lbl_bg')
viz2[0, 3] = widgets.VBox([oth_num_vssels_seen_text, oth_num_vssels_seen]).add_class('lbl_bg')
viz2

GridspecLayout(children=(Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

## Read in Map 3 data

In [7]:
bucket = 'afv-scenario'
key = 'data-products/AIS/AIS_2015_01.parquet'
ais2015 = client.get_object(Bucket=bucket, Key=key)
ais2015 = pd.read_parquet(io.BytesIO(ais2015['Body'].read()))

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world.geometry = world.geometry.buffer(10)

In [8]:
def plot_vessel_route(change):
    
    # Clear map layers and update text
    m3.clear_layers()
    m3.add_layer(basemap_layer)
    number_sightings_text.value = "<center><b><br>Number of times " + str(ship_selector.value) + " has been seen:</b>"
    countries_sighted_near_text.value = "<center><b><br>" + str(ship_selector.value) + " was sighted near the following countries: </b>"
    
    # Subset ais data to selected vessels and sort by date
    cur = ais2015[ais2015['MMSI'] == ship_selector.value]
    cur = cur.sort_values(by='BaseDateTime')
    
    # Convert AIS dataframe to spatial
    cur['geometry'] = cur.apply(lambda row: shapely.geometry.Point(row.LON, row.LAT), axis=1)
    cur = gpd.GeoDataFrame(cur, geometry='geometry')
        
    # Center map and zoom
    m3.center = (ais2015['LAT'].mean(), ais2015['LON'].mean())
    m3.zoom = 4
    
    # Plot each vessel sighting
    count = 1
    for col, row in cur.iterrows():
        p = create_circle_marker(row)
        m3.add_layer(p)
        plotting_text.value = "Plotting vessel " + str(count) + " out of " + str(len(cur))
        count += 1
        
    # Update message
    plotting_text.value = "Done plotting route of " + str(ship_selector.value)
    number_sightings_num.value = "<center>" + str(len(cur))
    
    countries, c = ["<center>"], 0
    for poly in world.geometry:
        overlap = gpd.clip(cur, poly)
        if len(overlap) != 0:
            countries.append(world.name[c])
        c += 1
    
    countries = "<center>".join(countries)
    countries_sighted_near_num.value = countries



# Initialize map elements
year_select_text = widgets.HTML(value = "<b>Select the year to show FoV's for: </b>")
year_selector3 = widgets.Dropdown(options = [2015, 2016, 2017], description = 'Year: ')
month_selector3 = widgets.Dropdown(options = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December'], description = 'Month:')
day_selector3 = widgets.Dropdown(options = get_days_observer(year_selector2, month_selector2), description = 'Day: ')
dates3 = initialize_date_options(year_selector3, month_selector3, day_selector3)
datetime_slider3 = widgets.widgets.SelectionRangeSlider(options = dates3, index = (0, len(dates) - 1), disabled = False, layout={'width': '400px'})
ship_select_text = widgets.HTML(value = "<b>Choose a vessel to plot: </b>")
shiplist2015 = pd.DataFrame(ais2015['MMSI'].value_counts()).reset_index()
shiplist2015 = shiplist2015[shiplist2015['MMSI'] < 500]
shiplist2015 = shiplist2015['index'].unique()
ship_selector = widgets.Dropdown(options = shiplist2015, description = 'Year: ')

update_ship_map_button = widgets.Button(description = "Plot Vessel Route")
plotting_text = widgets.HTML(value = "")
number_sightings_text = widgets.HTML(value = "<center><b><br>Number of times " + str(ship_selector.value) + " has been seen:</b>")
number_sightings_num = widgets.HTML(value = "")
countries_sighted_near_text = widgets.HTML("<center><b><br>" + str(ship_selector.value) + " was sighted near the following countries: </b>")
countries_sighted_near_num = widgets.HTML(value = "")

# Initialize map
m3 = leaflet.Map(basemap = basemap_layer, center=(0, 0), zoom=1)

# Map observers
update_ship_map_button.on_click(plot_vessel_route)

# Create visualization & layout
viz3 = widgets.GridspecLayout(6, 3, height='700px', layout = widgets.Layout(grid_gap='15px 5px'))
viz3[1:4, 1:] = m3
viz3[1:4, 0] = widgets.VBox([ship_select_text, ship_selector, br, update_ship_map_button, plotting_text]).add_class('filter_box3')
viz3[0, 0] = widgets.VBox([]).add_class('lbl_bg')
viz3[0, 1] = widgets.VBox([number_sightings_text, number_sightings_num]).add_class('lbl_bg')
viz3[0, 2] = widgets.VBox([countries_sighted_near_text, countries_sighted_near_num]).add_class('lbl_bg')
viz3

GridspecLayout(children=(Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_i…

## Create Main Viz

In [9]:
# Tab Specs
titles = ['Geographic Range of Satellites', 'Identify Vessels in View of Satellites', 'Vessel Routes']
children = [viz1, viz2, viz3]

# Initialize visualization
viz = widgets.Tab()

# Set Tab Specs
viz.children = children
for i in range(0, 3):
    viz.set_title(i, titles[i])

viz